In [55]:
import sys
import os
import pandas as pd
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from argparse import Namespace
from constants.paths import FOLDER_PATH
from examples.benchmark import local_get_args
from load_inputs.bike_in import load_data, START, END, DATE_COL, LOCATION_COL, VALUE_COL ,NAME,FILE_PATTERN,DATA_SUBFOLDER,DIRECTION



coverage_period = pd.date_range(start=START, end=END, freq='15min')[:-1]  # Exclude the last date to match the data
invalid_dates = []
minmaxnorm = True
standardize = False  # Set to True if you want to standardize the data
config = {'model_name': 'STGCN',
         'freq' : '15min',
        'dataset_names':['bike_in'],
        'dataset_for_coverage' : ['bike_in'],
        'target_data': 'bike_in',
        'contextual_kwargs':{'bike_in':{'agg_iris_target_n':50,
                                        'threshold_volume_min': 1,
                                },

                                        },
        'step_ahead':1,
        'horizon_step':1,
        }

args = local_get_args(config['model_name'],
                args_init = None,
                dataset_names=config['dataset_names'],
                dataset_for_coverage=config['dataset_for_coverage'],
                modification = config)


ds = load_data(FOLDER_PATH, coverage_period, invalid_dates, args, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None,
              file_pattern = FILE_PATTERN,
              data_subfolder = DATA_SUBFOLDER,
              date_col = DATE_COL,
              location_col = LOCATION_COL,
              value_col = VALUE_COL,
              direction = DIRECTION,
              name = NAME)

Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_attracted_by_station15min.csv...
df pivoted:  (70094, 435)
df reindexed :  (35040, 435)
Len coverage period:  35040
df filtered:  (35040, 435)
   Loaded data: (35040, 435)
   Dimension after spatial agg: (35040, 13)


In [61]:
from plotting.TS_analysis import plot_TS

df = pd.DataFrame(ds.raw_values,columns = ds.spatial_unit)
plot_TS(df.iloc[:2000,:],bool_show = True)

Loading BokehJS ...

figure(id='p1536', ...)